Add when wrong

```
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
```

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Automate

/content/drive/MyDrive/Automate


In [ ]:
!pip install tensorflow-gpu

In [5]:
# This will allow the notebook to run faster
from pathlib import Path
from general_utils import get_step2_prerequisite_files, read_training_files
from keras.utils import get_file
OUTPUT_PATH = Path('./data/seq2seq/')
OUTPUT_PATH.mkdir(exist_ok=True)

In [4]:
!pip install wget nmslib pathos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.4 MB 5.0 MB/s 
     |████████████████████████████████| 79 kB 9.8 MB/s 
     |████████████████████████████████| 188 kB 82.7 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 132 kB 83.3 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=be73b6e988d71e94dd3dbea647015760c6e91800cffe1358ea72fbb9ef95fcd1
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


# Read Text From File



In [6]:
# you want to supply the directory where the files are from step 1.
train_code, holdout_code, train_comment, holdout_comment = read_training_files('./data/processed_data/')

In [7]:
# code and comment files should be of the same length.

assert len(train_code) == len(train_comment)
assert len(holdout_code) == len(holdout_comment)

# Tokenize Text

In this step, we are going to pre-process the raw text for modeling.  For an explanation of what this section does, see the [Preapre & Clean Data section of this Tutorial](https://towardsdatascience.com/how-to-create-data-products-that-are-magical-using-sequence-to-sequence-models-703f86a231f8)

In [12]:
from ktext.preprocess import processor

code_proc = processor(heuristic_pct_padding=.7, keep_n=20000)
t_code = code_proc.fit_transform(train_code)

comment_proc = processor(append_indicators=True, heuristic_pct_padding=.7, keep_n=14000, padding ='post')
t_comment = comment_proc.fit_transform(train_comment)


 See full histogram by insepecting the `document_length_stats` attribute.
 See full histogram by insepecting the `document_length_stats` attribute.


In [ ]:
!pip install textacy==0.6.1

In [11]:
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install Keras==2.1.6

**Save tokenized text** (You will reuse this for step 4)

In [13]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open(OUTPUT_PATH/'py_code_proc_v2.dpkl', 'wb') as f:
    dpickle.dump(code_proc, f)

with open(OUTPUT_PATH/'py_comment_proc_v2.dpkl', 'wb') as f:
    dpickle.dump(comment_proc, f)

# Save the processed data
np.save(OUTPUT_PATH/'py_t_code_vecs_v2.npy', t_code)
np.save(OUTPUT_PATH/'py_t_comment_vecs_v2.npy', t_comment)

Arrange data for modeling

In [16]:
%reload_ext autoreload
%autoreload 2
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor


encoder_input_data, encoder_seq_len = load_encoder_inputs(OUTPUT_PATH/'py_t_code_vecs_v2.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs(OUTPUT_PATH/'py_t_comment_vecs_v2.npy')
num_encoder_tokens, enc_pp = load_text_processor(OUTPUT_PATH/'py_code_proc_v2.dpkl')
num_decoder_tokens, dec_pp = load_text_processor(OUTPUT_PATH/'py_comment_proc_v2.dpkl')

Shape of encoder input: (267630, 55)
Shape of decoder input: (267630, 14)
Shape of decoder target: (267630, 14)
Size of vocabulary for data/seq2seq/py_code_proc_v2.dpkl: 20,002
Size of vocabulary for data/seq2seq/py_comment_proc_v2.dpkl: 14,002


In [15]:
!pip install annoy==1.11.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 632 kB 5.0 MB/s 
  Created wheel for annoy: filename=annoy-1.11.5-cp38-cp38-linux_x86_64.whl size=303171 sha256=a44eaa113f6c1b2fccd617334558c611cd0a027830dd292029a609838db5e578
  Stored in directory: /root/.cache/pip/wheels/42/08/67/145506e4a49c72863367f7c4c2706e8e3da0841d211ddc470d
Successfully built annoy


In [ ]:
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Build Seq2Seq Model For Summarizing Code

We will build a model to predict the docstring given a function or a method.   The motivation for training this model is to learn a general purpose feature extractor for code that we can use for the task of code search.

In [17]:
from seq2seq_utils import build_seq2seq_model

The convenience function `build_seq2seq_model` constructs the architecture for a sequence-to-sequence model.  

The architecture built for this tutorial is a minimal example with only one layer for the encoder and decoder, and does not include things like [attention](https://nlp.stanford.edu/pubs/emnlp15_attn.pdf).  We encourage you to try and build different architectures to see what works best for you!

In [18]:
seq2seq_Model = build_seq2seq_model(word_emb_dim=800,
                                    hidden_state_dim=1000,
                                    encoder_seq_len=encoder_seq_len,
                                    num_encoder_tokens=num_encoder_tokens,
                                    num_decoder_tokens=num_decoder_tokens)

In [19]:
seq2seq_Model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Decoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Decoder-Word-Embedding (Embedd  (None, None, 800)   11201600    ['Decoder-Input[0][0]']          
 ing)                                                                                             
                                                                                                  
 Encoder-Input (InputLayer)     [(None, 55)]         0           []                               
                                                                                                  
 Decoder-Batchnorm-1 (BatchNorm  (None, None, 800)   3200        ['Decoder-Word-Embedding[0][0

### Train Seq2Seq Model

In [20]:
from keras.models import Model, load_model
import pandas as pd
import logging


from keras.callbacks import CSVLogger, ModelCheckpoint
import numpy as np
from keras import optimizers

seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.00005), loss='sparse_categorical_crossentropy')

script_name_base = 'py_func_sum_v9_'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))

model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                    save_best_only=True)

batch_size = 1500
epochs = 20
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

Epoch 1/20


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/nadam.py:78: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


158/158 [==============================] - 305s 2s/step - loss: 5.6072 - val_loss: 9.3276
Epoch 2/20
158/158 [==============================] - 296s 2s/step - loss: 4.0796 - val_loss: 7.7645
Epoch 3/20
158/158 [==============================] - 296s 2s/step - loss: 3.8630 - val_loss: 5.8234
Epoch 4/20
158/158 [==============================] - 297s 2s/step - loss: 3.7108 - val_loss: 3.9240
Epoch 5/20
158/158 [==============================] - 297s 2s/step - loss: 3.5778 - val_loss: 3.6118
Epoch 6/20
158/158 [==============================] - 297s 2s/step - loss: 3.4598 - val_loss: 3.5244
Epoch 7/20
158/158 [==============================] - 297s 2s/step - loss: 3.3580 - val_loss: 3.4386
Epoch 8/20
158/158 [==============================] - 295s 2s/step - loss: 3.2670 - val_loss: 3.4658
Epoch 9/20
158/158 [==============================] - 297s 2s/step - loss: 3.1860 - val_loss: 3.3415
Epoch 10/20
158/158 [==============================] - 298s 2s/step - loss: 3.1128 - val_loss: 3.2916


In [ ]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(139472, 55)
(139472, 14)
(139472, 14)


# Evaluate Seq2Seq Model For Code Summarization

To evaluate this model we are going to do two things:

1.  Manually inspect the results of predicted docstrings for code snippets, to make sure they look sensible.
2.  Calculate the [BLEU Score](https://en.wikipedia.org/wiki/BLEU) so that we can quantitately benchmark different iterations of this algorithm and to guide hyper-parameter tuning.

### Manually Inspect Results (on holdout set)

In [21]:
from seq2seq_utils import Seq2Seq_Inference
import pandas as pd

seq2seq_inf = Seq2Seq_Inference(encoder_preprocessor=enc_pp,
                                 decoder_preprocessor=dec_pp,
                                 seq2seq_model=seq2seq_Model)

demo_testdf = pd.DataFrame({'code':holdout_code, 'comment':holdout_comment, 'ref':''})
seq2seq_inf.demo_model_predictions(n=15, df=demo_testdf)



============== Example # 8432 =================

Original Input:
 def test_dependency_import self try import testpackage assert True except ImportError self fail depenency not installed
 

Original Output:
 sample test method for dependencies .

1/1 [==============================] - 0s 16ms/step

****** Predicted Output ******:
 test that import import works as expected


============== Example # 30681 =================

Original Input:
 def results self self log info Generating nodecon results from 0 policy format self self log debug Network 0 network r overlap 0 network_overlap format self self log debug IP Version 0 ip_version r format self self _match_context_debug self log for nodecon in self policy nodecons if self network if self network_overlap if not self network overlaps nodecon network continue elif not nodecon network self network continue if self ip_version and self ip_version nodecon ip_version continue if not self _match_context nodecon context continue yield nodecon


### Comment on manual inspection of results:

The predicted code summaries are not perfect, but we can see that the model has learned to extract some semantic meaning from the code.  That's all we need to get reasonable results in this case.  

### Calculate BLEU Score (on holdout set)

BLEU Score is described [in this wikipedia article](https://en.wikipedia.org/wiki/BLEU), and is a way to measure the efficacy of summarization/translation such as the one we conducted here.  This metric is useful if you wish to conduct extensive hyper-parameter tuning and try to improve the seq2seq model.

In [ ]:
# This will return a BLEU Score
seq2seq_inf.evaluate_model(input_strings=holdout_code, 
                           output_strings=holdout_comment, 
                           max_len=None)

# Save model to disk

Save the model to disk for part 4.

In [22]:
seq2seq_Model.save(OUTPUT_PATH/'code_summary_seq2seq_model.h5')